<a href="https://colab.research.google.com/github/jhackisneros/VISION_ARTIFICIAL/blob/main/12_explainability_gradcam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VISION_ARTIFICIAL - Notebook 12: Explainability con Grad-CAM

🔹 Importar librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import cv2
import os


🔹 Cargar modelo CNN entrenado

In [ ]:
model_path = "../models/cnn_best.keras"
model = load_model(model_path)
model.summary()


🔹 Cargar datos de test

In [ ]:
processed_dir = "../data/processed/"
x_test = np.load(os.path.join(processed_dir, "x_test.npy"))
y_test = np.load(os.path.join(processed_dir, "y_test.npy"))

🔹 Función Grad-CAM

In [ ]:
def get_gradcam(img_array, model, layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        class_idx = np.argmax(predictions[0])
        loss = predictions[:, class_idx]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = np.maximum(heatmap, 0) / (np.max(heatmap) + 1e-8)
    return heatmap, class_idx


🔹 Mostrar Grad-CAM sobre imagen

In [ ]:
def display_gradcam(img, heatmap, alpha=0.4):
    heatmap = cv2.resize(heatmap.numpy(), (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * alpha + np.uint8(img*255)
    plt.imshow(superimposed_img.astype(np.uint8))
    plt.axis('off')
    plt.show()


🔹 Elegir una imagen de prueba

In [ ]:
img = x_test[0][np.newaxis, ...]
heatmap, class_idx = get_gradcam(img, model, layer_name="conv2d_2")
print(f"Predicción clase: {class_idx}")
display_gradcam(x_test[0], heatmap)